## 📋 Setup & Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install wandb 

In [2]:
import os
import json
import torch
import wandb
from datasets import Dataset, load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

# Check GPU
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-28 08:30:50.823339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764318651.280937      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764318651.398819      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
GPU Available: True
GPU Name: Tesla T4
GPU Memory: 15.83 GB


## 🔐 WandB Login (for monitoring)

In [4]:
# Login to WandB for experiment tracking
# Get WandB API key from Kaggle Secrets
# In Kaggle: Add-ons → Secrets → Add new secret with key "WANDB_API_KEY"
# Get your API key from: https://wandb.ai/authorize

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Login with API key from Kaggle Secrets
wandb.login(key=wandb_api_key)

# Initialize WandB project with detailed config
wandb.init(
    project="vietnamese-legal-ai",
    name="llama3.2-3b-traffic-law-v1",
    config={
        "model": "meta-llama/Llama-3.2-3B-Instruct",
        "dataset": "traffic_law_data.jsonl",
        "task": "legal_qa",
        "language": "vietnamese",
        "max_seq_length": 1536,
        "lora_r": 32,
        "lora_alpha": 32,
        "learning_rate": 2e-4,
        "num_epochs": 2,
        "batch_size": 4,
        "gradient_accumulation": 4,
        "effective_batch_size": 32,
    },
    settings=wandb.Settings(
        _disable_meta=False,
        _disable_stats=False,
    )
)

print("✅ WandB initialized with detailed logging")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mikeethanh04 (mikeethanh04-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ WandB initialized with detailed logging


## ⚙️ Model Configuration

### Tại sao chọn Llama-3.2-3B-Instruct?
- ✅ **3B parameters**: Vừa đủ mạnh, vừa tiết kiệm GPU
- ✅ **Multilingual support**: Hỗ trợ nhiều ngôn ngữ bao gồm tiếng Việt
- ✅ **Instruct version**: Đã được train theo instruction format
- ✅ **Fit Kaggle T4**: ~15GB VRAM với 4-bit quantization
- ✅ **Unsloth optimized**: Hỗ trợ tốt, train nhanh 2x
- ✅ **Meta's latest**: Phiên bản mới nhất từ Meta (2024)

In [5]:
# Model configuration for Kaggle T4 (16GB VRAM)
max_seq_length = 1536  # Based on data analysis (covers 95% of samples)
dtype = None  # Auto-detect. Use Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage

# Alternative models (uncomment to try):
# model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"  # Qwen 2.5 3B
# model_name = "unsloth/gemma-2-2b-it-bnb-4bit"  # Gemma 2B (smaller, faster)
# model_name = "unsloth/Phi-3-mini-4k-instruct"  # Microsoft Phi-3

model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"  # Meta Llama 3.2 - Pre-quantized by Unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print(f"✅ Model loaded: {model_name}")
print(f"📏 Max sequence length: {max_seq_length}")
print(f"🔢 4-bit quantization: {load_in_4bit}")

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ Model loaded: unsloth/Llama-3.2-3B-Instruct-bnb-4bit
📏 Max sequence length: 1536
🔢 4-bit quantization: True


## 🎯 LoRA Configuration

### LoRA Parameters Explained:
- **r (rank)**: 16-32 cho balance quality/speed. Higher = better but slower
- **lora_alpha**: Scaling factor, thường = r hoặc 2*r
- **target_modules**: Train all attention & MLP layers cho best result
- **lora_dropout**: 0 cho faster training (Unsloth optimized)
- **bias**: "none" cho faster & less overfitting

In [6]:
# Apply LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # LoRA rank - higher = more expressive but slower (16, 32, 64)
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # All attention & MLP layers
    lora_alpha=32,  # LoRA scaling (usually = r or 2*r)
    lora_dropout=0,  # 0 is optimized by Unsloth
    bias="none",  # "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's long context support
    random_state=3407,  # For reproducibility
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None,  # LoftQ quantization
)

print("✅ LoRA adapters applied")
print(f"📊 Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"💡 Trainable ratio: {100 * sum(p.numel() for p in model.parameters() if p.requires_grad) / sum(p.numel() for p in model.parameters()):.2f}%")

Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


✅ LoRA adapters applied
📊 Trainable parameters: 48,627,712
📊 Total parameters: 1,852,091,392
💡 Trainable ratio: 2.63%


## 📊 Data Preparation

In [7]:
# Load data from Kaggle input (adjust path if uploading to Kaggle)
# For local testing, adjust the path
data_path = "/kaggle/input/traffic-law/traffic_law_data.jsonl"  # Kaggle path
# data_path = "../data/finetune_llm/traffic_law_data.jsonl"  # Local path

# Check if file exists
if not os.path.exists(data_path):
    print(f"⚠️ Data file not found at {data_path}")
    print("For Kaggle: Upload dataset or adjust path")
    print("For local: Make sure you're in the correct directory")
else:
    print(f"✅ Found data at: {data_path}")

# Load JSONL data
data = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

print(f"📊 Total samples: {len(data):,}")

# Show sample
print("\n📝 Sample data:")
sample = data[0]
for key, value in sample.items():
    if key == 'output':
        print(f"{key}: {value[:200]}...")  # Truncate long output
    else:
        print(f"{key}: {value}")

✅ Found data at: /kaggle/input/traffic-law/traffic_law_data.jsonl
📊 Total samples: 8,652

📝 Sample data:
instruction: Trả lời câu hỏi pháp luật sau:
input: Thời hạn kéo dài việc tạm giữ tang vật, phương tiện có tính ngày nghỉ không?
output: Theo Điều 8 Luật Xử lý vi phạm hành chính 2012 quy định về cách tính thời gian, thời hạn, thời hiệu trong xử lý vi phạm hành chính như sau: Cách tính thời gian, thời hạn, thời hiệu trong xử lý vi phạm...
domains: Hình sự, Dân sự, Giao thông
traffic_type: Vi phạm giao thông
output_words: 194
complexity: Unknown


In [8]:
# Split data: 90% train, 5% validation, 5% test
train_data, temp_data = train_test_split(data, test_size=0.1, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"📊 Train: {len(train_data):,} samples")
print(f"📊 Validation: {len(val_data):,} samples")
print(f"📊 Test: {len(test_data):,} samples")

📊 Train: 7,786 samples
📊 Validation: 433 samples
📊 Test: 433 samples


## 📝 Chat Template

Sử dụng chat template chuẩn của Llama 3.2 Instruct để đảm bảo tương thích với pretrained model:

In [9]:
from unsloth import apply_chat_template

# Chat template chuẩn cho Llama 3.2 Instruct
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""

def format_data_for_chat_template(examples):
    """Format data for Llama 3.2 chat template"""
    conversations = []
    
    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        # Combine instruction and input as the user message
        user_message = f"{instruction}\n\n{input_text}" if input_text else instruction
        
        conversation = [
            {"role": "system", "content": "Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Hãy trả lời chính xác và chi tiết các câu hỏi về luật giao thông."},
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": output}
        ]
        conversations.append(conversation)
    
    return {"conversations": conversations}

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

# Format data for chat template
train_dataset = train_dataset.map(format_data_for_chat_template, batched=True)
val_dataset = val_dataset.map(format_data_for_chat_template, batched=True)

# Apply chat template using Unsloth
train_dataset = apply_chat_template(
    train_dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    default_system_message="Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Hãy trả lời chính xác và chi tiết các câu hỏi về luật giao thông."
)

val_dataset = apply_chat_template(
    val_dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    default_system_message="Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Hãy trả lời chính xác và chi tiết các câu hỏi về luật giao thông."
)

print("✅ Data formatted with Llama 3.2 chat template")
print("\n📝 Example formatted conversation:")
print(train_dataset[0]['text'][:500] + "...")

Map:   0%|          | 0/7786 [00:00<?, ? examples/s]

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

Map:   0%|          | 0/7786 [00:00<?, ? examples/s]

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

✅ Data formatted with Llama 3.2 chat template

📝 Example formatted conversation:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Hãy trả lời chính xác và chi tiết các câu hỏi về luật giao thông.<|eot_id|><|start_header_id|>user<|end_header_id|>

Trả lời câu hỏi pháp luật sau:

Tài sản nào của cá nhân không được tiến hành thi hành án?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Căn cứ quy định khoản 2 Điều 87 Luật Thi hành án dân sự 2008 quy định về tài sản của cá nhân không được tiến hành thi h...


## 🎓 Training Configuration

### Optimized for Kaggle T4 (30h/week limit):
- **Epochs**: 3 (sufficient for legal domain)
- **Batch size**: 4 per device (max for T4 16GB)
- **Gradient accumulation**: 4 steps (effective batch = 16)
- **Learning rate**: 2e-4 (standard for LoRA)
- **Warmup**: 10% of steps
- **FP16**: Enabled for speed
- **Gradient checkpointing**: Unsloth optimized

In [10]:
# Training arguments optimized for Kaggle T4
training_args = TrainingArguments(
    # Output & Logging
    output_dir="./outputs",
    run_name="llama3.2-3b-traffic-law-v1",
    
    # Training dynamics
    num_train_epochs=2,  # 2-3 epochs is usually enough
    per_device_train_batch_size=4,  # Reduced from 4 to better utilize VRAM
    gradient_accumulation_steps=4,  # Increased to maintain effective batch size = 16
    
    # Optimization
    optim="adamw_8bit",  # 8-bit AdamW for memory efficiency
    learning_rate=2e-4,  # Standard for LoRA fine-tuning
    weight_decay=0.01,
    warmup_ratio=0.1,  # 10% warmup
    lr_scheduler_type="cosine",  # Cosine annealing
    
    # Performance
    fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 for T4
    bf16=torch.cuda.is_bf16_supported(),  # Use BF16 if supported (A100, H100)
    
    # Logging & Saving (more frequent for better monitoring)
    logging_steps=5,  # Log every 5 steps for better visibility
    logging_strategy="steps",
    logging_first_step=True,  # Log first step
    save_strategy="steps",
    save_steps=50,  # Save more frequently
    save_total_limit=3,  # Keep only 3 best checkpoints
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=50,  # Evaluate more frequently
    eval_accumulation_steps=1,  # Accumulate eval predictions
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # WandB integration with detailed logging
    report_to="wandb",
    logging_nan_inf_filter=True,  # Filter out NaN/Inf values
    include_inputs_for_metrics=False,  # Don't log inputs (save space)
    
    # Progress bar and output control
    disable_tqdm=False,  # Enable progress bar
    log_level="info",  # Show info messages
    log_level_replica="warning",
    log_on_each_node=True,
    dataloader_num_workers=2,
)

print("✅ Training arguments configured")
print(f"💾 Per device batch size: {training_args.per_device_train_batch_size}")
print(f"📊 Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"📈 Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"📊 Logging every {training_args.logging_steps} steps")
print(f"📊 Evaluating every {training_args.eval_steps} steps")
print(f"🕐 Total training steps: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

✅ Training arguments configured
💾 Per device batch size: 4
📊 Effective batch size: 16
📈 Gradient accumulation: 4
📊 Logging every 5 steps
📊 Evaluating every 50 steps
🕐 Total training steps: 972


In [12]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences
    args=training_args,
)

print("✅ Trainer initialized")

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/7786 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/433 [00:00<?, ? examples/s]

Using auto half precision backend


✅ Trainer initialized


## 🚀 Start Training!

**Estimated time on T4**: ~3-4 hours for 3 epochs  
**Memory usage**: ~14-15GB VRAM  
**Kaggle time budget**: ~4h / 30h week (leaves 26h for experiments)

In [13]:
# Show GPU stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"🖥️ GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"💾 {start_gpu_memory} GB of memory reserved.")

# Start training
print("\n🚀 Starting training...\n")
trainer_stats = trainer.train()

# Show final stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)

print("\n" + "="*50)
print("✅ TRAINING COMPLETED!")
print("="*50)
print(f"⏱️ Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"💾 Peak reserved memory: {used_memory} GB")
print(f"📊 Memory used for training: {used_memory_for_lora} GB")
print(f"📈 Percentage of max memory: {used_percentage}%")
print(f"🎯 Final train loss: {trainer_stats.metrics['train_loss']:.4f}")

🖥️ GPU = Tesla T4. Max memory = 14.741 GB.
💾 3.07 GB of memory reserved.

🚀 Starting training...



The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, domains, output_words, instruction, traffic_type, output, text, attention_mask, conversations, complexity. If input, domains, output_words, instruction, traffic_type, output, text, attention_mask, conversations, complexity are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
skipped Embedding(128256, 3072, padding_idx=128004): 375.75M params
skipped: 375.75M params
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,786 | Num Epochs = 2 | Total steps = 488
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLE

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,1.442100,1.378838
100,1.231800,1.208347
150,1.194700,1.139060
200,1.101200,1.090938
250,1.027300,1.052491
300,0.982300,1.024429
350,0.953000,1.002104
400,0.985600,0.989309
450,0.970900,0.984937


The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, domains, output_words, instruction, traffic_type, output, text, attention_mask, conversations, complexity. If input, domains, output_words, instruction, traffic_type, output, text, attention_mask, conversations, complexity are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 433
  Batch size = 8
Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Saving model checkpoint to ./outputs/checkpoint-50
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3.2-3b-instruct-bnb-4bit/snapshots/bb1d317a108579fb40e646af8924a5e7ec5604b1/config.json
Mo

eval/loss,█▅▄▃▂▂▁▁▁
eval/runtime,█▃▄▄▁▅▅▆▂
eval/samples_per_second,▁▆▅▅█▅▄▄▇
eval/steps_per_second,▁▅▅▅█▅▅▅█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████
train/grad_norm,█▂▂▁▂▄▁▂▁▂▁▂▁▃▂▂▂▂▂▂▁▂▂▂▂▂▂▂▃▂▃▂▃▂▃▂▂▃▂▂
train/learning_rate,▁▃▄▄▇██████▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,█▅▅▅▅▄▄▃▃▄▃▃▃▃▃▂▃▃▃▂▂▁▂▂▁▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁
eval/loss,0.98494
eval/runtime,217.7863



✅ TRAINING COMPLETED!
⏱️ Training time: 17256.84 seconds
💾 Peak reserved memory: 11.223 GB
📊 Memory used for training: 8.153 GB
📈 Percentage of max memory: 76.135%
🎯 Final train loss: 1.1240


## 📊 Evaluation

In [14]:
# Evaluate on validation set
print("📊 Evaluating on validation set...\n")
eval_results = trainer.evaluate()

print("="*50)
print("VALIDATION RESULTS")
print("="*50)
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")

# Log to WandB
wandb.log({"final_eval_loss": eval_results['eval_loss']})

The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, domains, output_words, instruction, traffic_type, output, text, attention_mask, conversations, complexity. If input, domains, output_words, instruction, traffic_type, output, text, attention_mask, conversations, complexity are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 433
  Batch size = 8


📊 Evaluating on validation set...



Error: You must call wandb.init() before wandb.log()

## 🧪 Inference Testing

In [15]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

def test_model(instruction, input_text, max_new_tokens=512):
    """Test model with chat template format"""
    # Create conversation format
    user_message = f"{instruction}\n\n{input_text}" if input_text else instruction
    
    messages = [
        {"role": "system", "content": "Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Hãy trả lời chính xác và chi tiết các câu hỏi về luật giao thông."},
        {"role": "user", "content": user_message}
    ]
    
    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.batch_decode(outputs)[0]
    # Extract only the response part (after the last assistant header)
    if "<|start_header_id|>assistant<|end_header_id|>" in response:
        response = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        response = response.split("<|eot_id|>")[0].strip()
    
    return response

# Test with samples from test set
print("🧪 Testing model on random samples...\n")
print("="*80)

import random
test_samples = random.sample(test_data, 3)

for i, sample in enumerate(test_samples, 1):
    print(f"\n{'='*80}")
    print(f"TEST SAMPLE #{i}")
    print(f"{'='*80}")
    print(f"\n📝 Instruction: {sample['instruction']}")
    print(f"\n❓ Input: {sample['input']}")
    print(f"\n🎯 Expected Output:\n{sample['output'][:300]}...")
    
    # Generate response
    response = test_model(sample['instruction'], sample['input'])
    print(f"\n🤖 Model Response:\n{response}")
    print(f"\n{'='*80}")

🧪 Testing model on random samples...


TEST SAMPLE #1

📝 Instruction: Trả lời câu hỏi pháp luật sau:

❓ Input: Các hành vi nào bị cấm trong hoạt động quảng cáo?

🎯 Expected Output:
Theo quy định tại Điều 8 Luật Quảng cáo 2012 thì các hành vi cấm trong hoạt động quảng cáo bao gồm: - Quảng cáo những sản phẩm, hàng hóa, dịch vụ quy định tại Điều 7 Luật Quảng cáo 2012. - Quảng cáo làm tiết lộ bí mật nhà nước, phương hại đến độc lập, chủ quyền quốc gia, an ninh, quốc phòng. - Quảng...

🤖 Model Response:
Căn cứ Điều 8 Luật Quảng cáo 2012 quy định các hành vi bị cấm trong hoạt động quảng cáo như sau: - Quảng cáo có nội dung xâm phạm lợi ích quốc gia, lợi ích công cộng, an ninh, quốc phòng, trật tự, an toàn xã hội, nhân phẩm, danh dự, nhân phẩm, uy tín của tổ chức, danh dự, nhân phẩm, uy tín của cá nhân. - Quảng cáo có nội dung có tính chất đe dọa, xúc phạm uy tín của cơ quan, tổ chức, danh dự, nhân phẩm, uy tín của cá nhân. - Quảng cáo có tính chất kích động bạo lực, bạo lực, tội phạm. - Quản

## 💾 Save Model

In [16]:
# Save LoRA adapters (only ~100-200MB!)
#model.save_pretrained("vietnamese_legal_lora")
#tokenizer.save_pretrained("vietnamese_legal_lora")

#rint("✅ LoRA adapters saved to: vietnamese_legal_lora/")
#print("📦 Size: ~100-200MB (adapters only)")

# Optional: Save merged model (full size ~6GB)
model.save_pretrained_merged("vietnamese_legal_merged", tokenizer, save_method="merged_16bit")
print("✅ Merged model saved to: vietnamese_legal_merged/")

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Configuration saved in vietnamese_legal_merged/config.json


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [00:22<00:00, 11.41s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:41<00:00, 20.57s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/vietnamese_legal_merged`
✅ Merged model saved to: vietnamese_legal_merged/


In [ ]:
import os
import shutil
from google.colab import files

# Detect environment
is_colab = "COLAB_" in "".join(os.environ.keys())
is_kaggle = os.path.exists("/kaggle/working")

if is_colab:
    print("🔍 Detected: Google Colab")
    print("📦 Compressing models for download...\n")
    
    from google.colab import files
    
    # Zip LoRA adapters
    if os.path.exists("vietnamese_legal_lora"):
        shutil.make_archive('vietnamese_legal_lora', 'zip', 'vietnamese_legal_lora')
        print("✅ LoRA adapters zipped: vietnamese_legal_lora.zip (~100-200MB)")
        files.download('vietnamese_legal_lora.zip')
    
    # Zip merged model (if exists)
if os.path.exists("vietnamese_legal_merged"):
        print("\n⚠️ Merged model is ~6GB, this may take a while...")
        zip_path = shutil.make_archive('vietnamese_legal_merged', 'zip', 'vietnamese_legal_merged')
        print(f"✅ Merged model zipped: {zip_path} (~6GB)")
        if files:
            files.download(zip_path)
        else:
            print(f"📂 File saved at: {os.path.abspath(zip_path)}")
    

elif is_kaggle:
    print("🔍 Detected: Kaggle")
    print("📂 Models are saved in the output directory.")
    print("ℹ️ After notebook execution completes:")
    print("  1. Go to 'Output' tab on the right")
    print("  2. Download the following folders:")
    print("     - vietnamese_legal_lora/ (~100-200MB)")
    print("     - vietnamese_legal_merged/ (~6GB, if saved)")
    print("     - *.gguf files (if exported)")
    print("\n💡 Tip: Models in the output directory will be available for 7 days.")

else:
    print("🔍 Detected: Local environment")
    print("📂 Models saved at:")
    
    if os.path.exists("vietnamese_legal_lora"):
        lora_path = os.path.abspath("vietnamese_legal_lora")
        print(f"  ✅ LoRA adapters: {lora_path}")
    
    if os.path.exists("vietnamese_legal_merged"):
        merged_path = os.path.abspath("vietnamese_legal_merged")
        print(f"  ✅ Merged model: {merged_path}")
    
    gguf_files = [f for f in os.listdir('.') if f.endswith('.gguf')]
    if gguf_files:
        print(f"  ✅ GGUF models: {os.path.abspath('.')}")
        for gguf_file in gguf_files:
            print(f"     - {gguf_file}")
    
    print("\n✅ Models are already on your local machine!")

In [19]:
# Upload merged model to HuggingFace Hub (giải pháp cho file lớn!)
# Bước 1: Tạo HuggingFace account tại https://huggingface.co/join
# Bước 2: Tạo token tại https://huggingface.co/settings/tokens (chọn "Write" permission)
# Bước 3: Thêm token vào Kaggle Secrets với key "HF_TOKEN"

import os

if os.path.exists("/kaggle/working"):
    print("🚀 Uploading model to HuggingFace Hub...")
    print("="*70)
    
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        hf_token = user_secrets.get_secret("HF_TOKEN")
        
        from huggingface_hub import HfApi, login
        
        # Login to HuggingFace
        login(token=hf_token)
        print("✅ Logged in to HuggingFace")
        
        # Thay YOUR_USERNAME bằng username HuggingFace của bạn
        YOUR_HF_USERNAME = "mikeethanh"  # ⚠️ SỬA DÒNG NÀY!
        repo_name = f"{YOUR_HF_USERNAME}/vietnamese-legal-llama3.2-3b-merged-sft-v1"
        
        print(f"\n📤 Uploading to: {repo_name}")
        print("⏳ Đang upload ~6GB, có thể mất 10-15 phút...\n")
        
        # Upload merged model
        if os.path.exists("vietnamese_legal_merged"):
            from huggingface_hub import create_repo, upload_folder
            
            # Create repo (public)
            try:
                create_repo(repo_name, repo_type="model", exist_ok=True)
                print(f"✅ Repository created: https://huggingface.co/{repo_name}")
            except:
                print(f"ℹ️ Repository already exists: https://huggingface.co/{repo_name}")
            
            # Upload folder
            upload_folder(
                folder_path="vietnamese_legal_merged",
                repo_id=repo_name,
                commit_message="Vietnamese Legal AI - Llama 3.2 3B Merged Model",
            )
            
            print("\n" + "="*70)
            print("✅ UPLOAD THÀNH CÔNG!")
            print("="*70)
            print(f"\n📥 Download model về máy bằng cách:")
            print(f"   git clone https://huggingface.co/{repo_name}")
            print(f"\n🌐 Hoặc xem trên web:")
            print(f"   https://huggingface.co/{repo_name}")
            print("\n💡 Model đã public, ai cũng có thể download!")
        else:
            print("⚠️ Folder 'vietnamese_legal_merged' not found!")
            
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n📝 Hướng dẫn fix:")
        print("  1. Tạo account tại: https://huggingface.co/join")
        print("  2. Tạo token tại: https://huggingface.co/settings/tokens")
        print("  3. Kaggle: Add-ons → Secrets → Add 'HF_TOKEN'")
        print("  4. Sửa YOUR_USERNAME trong code")
        
else:
    print("ℹ️ This cell only works on Kaggle")
    print("💡 For local, use: model.push_to_hub() directly")

🚀 Uploading model to HuggingFace Hub...
✅ Logged in to HuggingFace

📤 Uploading to: mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1
⏳ Đang upload ~6GB, có thể mất 10-15 phút...

✅ Repository created: https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


✅ UPLOAD THÀNH CÔNG!

📥 Download model về máy bằng cách:
   git clone https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1

🌐 Hoặc xem trên web:
   https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1

💡 Model đã public, ai cũng có thể download!


## 📤 Push to HuggingFace Hub (Optional)

In [ ]:
# Uncomment to push to HuggingFace Hub
# You need to login first: huggingface-cli login

# model.push_to_hub(
#     "your-username/vietnamese-legal-llama3.2-3b-lora",
#     token="your_hf_token",
#     commit_message="Vietnamese Legal AI - Traffic Law QA"
# )
# tokenizer.push_to_hub(
#     "your-username/vietnamese-legal-llama3.2-3b-lora",
#     token="your_hf_token"
# )

# print("✅ Model pushed to HuggingFace Hub!")

## 📊 Quantization Export (for deployment)

In [ ]:
# Export to GGUF for llama.cpp / Ollama deployment
# Uncomment the quantization method you want

quantization_methods = [
    "q8_0",    # Fast inference, good quality (recommended)
    # "q4_k_m",  # Smaller size, still good quality
    # "q5_k_m",  # Balance between size and quality
]

for method in quantization_methods:
    print(f"\n📦 Exporting to {method.upper()}...")
    model.save_pretrained_gguf(
        "vietnamese_legal_model",
        tokenizer,
        quantization_method=method,
    )
    print(f"✅ Exported: vietnamese_legal_model-{method.upper()}.gguf")

print("\n✅ All quantization exports completed!")
print("📝 You can now use these with Ollama or llama.cpp")

## 🎉 Finish & Cleanup

In [ ]:
# Finish WandB run
wandb.finish()

# Clear GPU memory
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

print("✅ Training completed successfully!")
print("\n📊 Summary:")
print(f"  - Model: Llama-3.2-3B-Instruct")
print(f"  - Training samples: {len(train_data):,}")
print(f"  - Validation samples: {len(val_data):,}")
print(f"  - Test samples: {len(test_data):,}")
print(f"  - Training time: ~{trainer_stats.metrics['train_runtime']/3600:.2f} hours")
print(f"  - Final eval loss: {eval_results['eval_loss']:.4f}")
print("\n📂 Saved outputs:")
print("  - LoRA adapters: vietnamese_legal_lora/")
print("  - GGUF models: vietnamese_legal_model-*.gguf")
print("\n🎯 Next steps:")
print("  1. Test model on more samples")
print("  2. Deploy with Ollama or llama.cpp")
print("  3. Collect feedback and iterate")

---

## 📚 References & Resources

- **Unsloth**: https://github.com/unslothai/unsloth
- **Unsloth Docs**: https://docs.unsloth.ai
- **WandB**: https://wandb.ai
- **Llama 3.2**: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

## 💡 Tips for Better Results

1. **More data**: Collect more Vietnamese legal Q&A pairs
2. **Data quality**: Clean and verify answers
3. **Hyperparameter tuning**: Try different learning rates (1e-4, 5e-5)
4. **Longer training**: Try 4-5 epochs if not overfitting
5. **Larger model**: Try Llama-3.2-11B if you have more GPU
6. **Domain adaptation**: Continue pretraining on legal documents first

## 🐛 Troubleshooting

- **OOM (Out of Memory)**: Reduce batch size or max_seq_length
- **Slow training**: Enable packing=True for short sequences
- **Poor results**: Increase LoRA rank or training epochs
- **Overfitting**: Reduce epochs or add more data augmentation